In [1]:
import os
import sys
os.chdir('..')
sys.path.append(os.getcwd())

In [2]:
import torch
from Models.pretrain import *
from Models.cPB import cPB
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
import argparse
from torch.autograd import Variable
import pprint
import copy
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=UserWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=DeprecationWarning, module='torch.storage')

# SINE datasets


## Executable code:


In [ ]:
hidden_size_list = [150]
pretrain_mask_list = ['4']
for hidden in hidden_size_list:
  for pretrain in pretrain_mask_list:
    for iter in range(1,11):
      ###### Config the parameters ######
      base_model = 'GRU'
      batch_size = 128
      hidden_size = hidden
      seq_len = 10
      epoch_size=10
      lr = 0.01
      number_of_tasks=4
      mask_selection_NofBatch = 50
      input_size=2
      iteration=iter
      mask_init='uniform'
      dataset='SINE'
      dataset_name = "sine_rw10_mode5_extended_16-16_2341"
      ###### End of config the parameters ######
      Pretrain_task = pretrain
      df = pd.read_csv(os.path.join(f"datasets/{dataset}/", f"{dataset_name}.csv"))
      pretrain_model_addr=f'Performance/Pretrain/{base_model}/{dataset}/After/{dataset}-Task_{Pretrain_task}-{base_model}-pretrain-hidden{hidden_size}-epoch10_iter{iteration}.pickle'
      mask_weights=[] #if we have initial masks then reload it here


      mask_option = 'SUM'
      model_type = 'cPB'
      batch_first = True
      low_rank = False
      model=cPB(lr = lr ,hidden_size = hidden_size, seq_len = seq_len,base_model = base_model, pretrain_model_addr = pretrain_model_addr,
                mask_weights = mask_weights, mask_init = mask_init, number_of_tasks = number_of_tasks, epoch_size = epoch_size,
                input_size = input_size, mask_option = mask_option, model_type = model_type, batch_first = batch_first, low_rank = low_rank)


      # loop for each task
      for task in range(1, df["task"].max() + 1):
        df_task = df[df["task"] == task]
        df_task = df_task.drop(columns="task")
        # loop based on each batch of data
        batch_cont=0
        model.weights_copy(task)
        for i in range(0, len(df_task), batch_size):
          x = df_task.iloc[i : i + batch_size, 0:-1].values.astype(np.float32)
          y = list(df_task.iloc[i : i + batch_size, -1])
          if batch_cont<mask_selection_NofBatch:
            for mask_index in range (0, task):
              model.predict_many(x,y,mask_index,task,mask_selection=True)
              model.learn_many(x,y,mask_index)
            batch_cont+=1
          elif batch_cont==mask_selection_NofBatch:
            best_mask_index=model.add_new_column(task)
            model.predict_many(x,y,best_mask_index,task,mask_selection=False)
            model.learn_many(x,y,best_mask_index)
            batch_cont+=1
          elif batch_cont>mask_selection_NofBatch:
            batch_cont+=1
            model.predict_many(x,y,best_mask_index,task,mask_selection=False)
            model.learn_many(x,y,best_mask_index)
        model.save_final_metrics(task,best_mask_index)
        model.final_weights_saving()
      model.plotting()


      file_path = f'Performance/Results/{model_type}/{dataset}/Periodic/pretrain-T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}_iter{iteration}.pkl'
      with open(file_path, 'wb') as file:
          pickle.dump(model.performance, file)

      file_path = f'Performance/Results/{model_type}/{dataset}/Periodic/pretrain-T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}_iter{iteration}_selectedMask.pkl'
      with open(file_path, 'wb') as file:
          pickle.dump(model.selected_mask_index, file)

      file_path=f'Performance/Final Models/{model_type}/{base_model}/{dataset}/Periodic/{dataset_name}-{base_model}-pretrain-hidden{hidden_size}-epoch10-Mask_{mask_option}-itter{iteration}.pickle'
      with open(file_path, 'wb') as file:
          pickle.dump(model.all_models_weight, file)

# Weather datasets


## Executable code:


In [ ]:
for iter in range(1,11):
  ###### Config the parameters ######
  base_model = 'GRU'
  batch_size = 128
  hidden_size = 50
  seq_len = 11
  epoch_size=10
  lr = 0.01
  number_of_tasks=4
  mask_selection_NofBatch = 50
  input_size=4
  iteration=iter
  mask_init='uniform'
  dataset='Weather'
  dataset_name = "weather_st124_4conf"
  Pretrain_task = '1'
  ###### End of config the parameters ######

  df = pd.read_csv(os.path.join(f"datasets/{dataset}/", f"{dataset_name}.csv"))
  pretrain_model_addr=f'Performance/Pretrain/{base_model}/{dataset}/After/{dataset}-{base_model}-pretrain-hidden{hidden_size}-epoch10_itter{iteration}.pickle'
  mask_weights=[] #if we have initial masks then reload it here

  mask_option = 'SUM'
  model_type = 'cPB'
  batch_first = True
  low_rank = False


  model = cPB(lr = lr ,hidden_size = hidden_size, seq_len = seq_len,base_model = base_model, pretrain_model_addr = pretrain_model_addr,
            mask_weights = mask_weights, mask_init = mask_init, number_of_tasks = number_of_tasks, epoch_size = epoch_size,
            input_size = input_size, mask_option = mask_option, model_type = model_type, batch_first = batch_first, low_rank = low_rank)


  for task in range(1, df["task"].max() + 1):
    df_task = df[df["task"] == task]
    df_task = df_task.drop(columns="task")
    # loop based on each batch of data
    batch_cont=0
    model.weights_copy(task)
    for i in range(0, len(df_task), batch_size):
      x = df_task.iloc[i : i + batch_size, 0:-1].values.astype(np.float32)
      y = list(df_task.iloc[i : i + batch_size, -1])
      if batch_cont<mask_selection_NofBatch:
        for mask_index in range (0, task):
          model.predict_many(x,y,mask_index,task,mask_selection=True)
          model.learn_many(x,y,mask_index)
        batch_cont+=1
      elif batch_cont==mask_selection_NofBatch:
        best_mask_index=model.add_new_column(task)
        model.predict_many(x,y,best_mask_index,task,mask_selection=False)
        model.learn_many(x,y,best_mask_index)
        batch_cont+=1
      elif batch_cont>mask_selection_NofBatch:
        batch_cont+=1
        model.predict_many(x,y,best_mask_index,task,mask_selection=False)
        model.learn_many(x,y,best_mask_index)
    model.save_final_metrics(task,best_mask_index)
    model.final_weights_saving()
  model.plotting()


  file_path = f'Performance/Results/{model_type}/{dataset}/Periodic/pretrain_T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}-iter{iteration}.pkl'
  with open(file_path, 'wb') as file:
      pickle.dump(model.performance, file)

  file_path = f'Performance/Results/{model_type}/{dataset}/Periodic/pretrain_T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}-iter{iteration}_selectedMask.pkl'
  with open(file_path, 'wb') as file:
      pickle.dump(model.selected_mask_index, file)
      
      
  file_path=f'Performance/Final Models/{model_type}/{base_model}/{dataset}/{dataset_name}-{base_model}-pretrain-hidden{hidden_size}-epoch10-Mask_{mask_option}-itter{iteration}.pickle'
  with open(file_path, 'wb') as file:
      pickle.dump(model.all_models_weight, file)
      
  print(f'itter {iteration} finished')

